# Imports

In [1]:
import webdriver_manager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager
from IPython.display import clear_output
import time
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
import os
import json
import vlc
import re
import tqdm
import numpy as np

# Getting Ready

In [2]:
#Open ChromeDriver
package = ChromeDriverManager().install()
driver = webdriver.Chrome(package)
# driver.get("https://www.amazon.in/s?k=Dune&i=stripbooks")



====== WebDriver manager ======
Current google-chrome version is 103.0.5060
Get LATEST chromedriver version for 103.0.5060 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/103.0.5060.53/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\harik\.wdm\drivers\chromedriver\win32\103.0.5060.53]
<ipython-input-2-4eb2002836ca>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(package)


In [7]:
books = pd.read_csv("Final_Books_With_Images.csv")
books.head()

,Book Name,Author,ISBN 13,ISBN,Number of Pages,Average Rating,Total Number of Ratings,Publication Date,Date Started,Date Completed,Name,Series,Volume,Goodreads Url,Description,Genre,Image Url,Low Links,High Links
0,The Lightning Tree,Patrick Rothfuss,NaN,NaN,93,4.00,2156,2014-01-01,2022-04-16,2022-04-16,The Lightning Tree,Standalone,1,https://www.goodreads.com/book/show/55262194,The Lightning Tree is a companion short story ...,"['Fantasy', 'Fiction', 'Short Stories', 'Fanta...",https://i.gr-assets.com/images/S/compressed.ph...,"{'1x': [], '1.5x': [], '2x': [], '2.5x': [], '...",[]
1,What If? Serious Scientific Answers to Absurd ...,Randall Munroe,9.780544e+12,0544272994,303,4.14,151303,2014-09-02,NaN,NaN,What If? Serious Scientific Answers to Absurd ...,What If?,1,https://www.goodreads.com/book/show/21413662,Randall Munroe left NASA in 2005 to start up h...,"['Nonfiction', 'Science', 'Humor', 'Audiobook'...",https://i.gr-assets.com/images/S/compressed.ph...,NaN,NaN
2,"Leviathan Falls (The Expanse, #9)",James S.A. Corey,9.780357e+12,0356510395,528,4.54,31066,2021-11-30,NaN,NaN,Leviathan Falls,The Expanse,9,https://www.goodreads.com/book/show/58783427,"The Laconian Empire has fallen, setting the th...","['Science Fiction', 'Fiction', 'Space', 'Space...",NaN,{'1x': ['https://m.media-amazon.com/images/I/8...,[]
3,"Tiamat's Wrath (The Expanse, #8)",James S.A. Corey,9.780316e+12,0316332879,534,4.57,47842,2019-03-26,NaN,NaN,Tiamat's Wrath,The Expanse,8,https://www.goodreads.com/book/show/28335698,Thirteen hundred gates have opened to solar sy...,"['Science Fiction', 'Fiction', 'Space', 'Space...",https://i.gr-assets.com/images/S/compressed.ph...,{'1x': ['https://m.media-amazon.com/images/I/8...,['https://images-na.ssl-images-amazon.com/imag...
4,"Persepolis Rising (The Expanse, #7)",James S.A. Corey,9.780316e+12,0316332828,608,4.37,53386,2017-12-05,2022-06-02,NaN,Persepolis Rising,The Expanse,7,https://www.goodreads.com/book/show/28335696,In the thousand-sun network of humanity's expa...,"['Science Fiction', 'Fiction', 'Space', 'Space...",https://i.gr-assets.com/images/S/compressed.ph...,{'1x': ['https://m.media-amazon.com/images/I/9...,['https://images-na.ssl-images-amazon.com/imag...


In [8]:
book_names = books['Name']
authors = books['Author']

In [9]:
print(f"{book_names[10]} {authors[10]}")

Caliban's War James S.A. Corey


# Getting Image Urls

## Using Amazon

In [33]:
def get_images_from_amazon(book, author, num = 3):
    link = "https://www.amazon.in/s?k=" + book+ " " + author + "&i=stripbooks"
    print("Opening Url")
    driver.get(link)
    print("Getting Image tags")
    
    i=0
    j=0
    raw_img_srcs = []
    high_image_urls = []
    max_tries = 10
    while True:
        image = driver.find_elements(By.CLASS_NAME, "s-image")[i]
        j+=1
        try:
            if book.lower()[:10] in image.get_attribute("alt").lower():
                print("Getting image srcs")
                raw_img_srcs.append(image.get_property("srcset"))
                i+=1
                print("Clicking on image")
                image.click()
                print("Switching to new window")
                driver.switch_to.window(driver.window_handles[-1])
                print("Clicking on thumb button")
                try:
                    driver.find_elements(By.CLASS_NAME, "thumb-text")[0].click()
                    print("Getting High Resolution Image")
                    for el in driver.find_elements(By.CLASS_NAME, "ig-thumb-image"):
                        el.click()
                        high_image_urls.append(driver.find_elements(By.ID, "igImage")[0].get_attribute("src"))
                except:
                    print("It did not worked. Trying the next image.")
                print("Closing the Extra Window")
                driver.close()
                print("Switching to the Main Window")
                driver.switch_to.window(driver.window_handles[0])
        except Exception as e:
            print(e)
            print("Error in getting image")
            if j>max_tries:
                break
        if i == num:
            break
        if j == max_tries:
            break
    return raw_img_srcs, high_image_urls

In [52]:
package = ChromeDriverManager().install()
driver = webdriver.Chrome(package)



====== WebDriver manager ======
Current google-chrome version is 103.0.5060
Get LATEST chromedriver version for 103.0.5060 google-chrome
Driver [C:\Users\harik\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe] found in cache
<ipython-input-52-2192dd8f43b5>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(package)


In [68]:
low, high = get_images_from_amazon(book_names[20], authors[20])

Opening Url
Getting Image tags
Getting image srcs
Clicking on image
Switching to new window
Clicking on thumb button
Getting High Resolution Image
Closing the Extra Window
Switching to the Main Window
Getting image srcs
Clicking on image
Switching to new window
Clicking on thumb button
It did not worked. Trying the next image.
Closing the Extra Window
Switching to the Main Window
Getting image srcs
Clicking on image
Switching to new window
Clicking on thumb button
Getting High Resolution Image
Closing the Extra Window
Switching to the Main Window


In [69]:
low , high

(['https://m.media-amazon.com/images/I/71XBl40bfiL._AC_UY218_.jpg 1x, https://m.media-amazon.com/images/I/71XBl40bfiL._AC_UY327_FMwebp_QL65_.jpg 1.5x, https://m.media-amazon.com/images/I/71XBl40bfiL._AC_UY436_FMwebp_QL65_.jpg 2x, https://m.media-amazon.com/images/I/71XBl40bfiL._AC_UY545_FMwebp_QL65_.jpg 2.5x, https://m.media-amazon.com/images/I/71XBl40bfiL._AC_UY654_FMwebp_QL65_.jpg 3x',
  'https://m.media-amazon.com/images/I/910TAjw6e7L._AC_UY218_.jpg 1x, https://m.media-amazon.com/images/I/910TAjw6e7L._AC_UY327_FMwebp_QL65_.jpg 1.5x, https://m.media-amazon.com/images/I/910TAjw6e7L._AC_UY436_FMwebp_QL65_.jpg 2x, https://m.media-amazon.com/images/I/910TAjw6e7L._AC_UY545_FMwebp_QL65_.jpg 2.5x, https://m.media-amazon.com/images/I/910TAjw6e7L._AC_UY654_FMwebp_QL65_.jpg 3x',
  'https://m.media-amazon.com/images/I/71XwWSxCseL._AC_UY218_.jpg 1x, https://m.media-amazon.com/images/I/71XwWSxCseL._AC_UY327_FMwebp_QL65_.jpg 1.5x, https://m.media-amazon.com/images/I/71XwWSxCseL._AC_UY436_FMwebp_QL

### Parsing the Low Resolution Image Links

In [100]:
urls = low[1]
urls

'https://m.media-amazon.com/images/I/910TAjw6e7L._AC_UY218_.jpg 1x, https://m.media-amazon.com/images/I/910TAjw6e7L._AC_UY327_FMwebp_QL65_.jpg 1.5x, https://m.media-amazon.com/images/I/910TAjw6e7L._AC_UY436_FMwebp_QL65_.jpg 2x, https://m.media-amazon.com/images/I/910TAjw6e7L._AC_UY545_FMwebp_QL65_.jpg 2.5x, https://m.media-amazon.com/images/I/910TAjw6e7L._AC_UY654_FMwebp_QL65_.jpg 3x'

In [79]:
web_regex = re.compile(r'https://.*?\.jpg|\.png|\.jpeg|.webp')
web_regex.findall(urls)

['https://m.media-amazon.com/images/I/71XBl40bfiL._AC_UY218_.jpg',
 'https://m.media-amazon.com/images/I/71XBl40bfiL._AC_UY327_FMwebp_QL65_.jpg',
 'https://m.media-amazon.com/images/I/71XBl40bfiL._AC_UY436_FMwebp_QL65_.jpg',
 'https://m.media-amazon.com/images/I/71XBl40bfiL._AC_UY545_FMwebp_QL65_.jpg',
 'https://m.media-amazon.com/images/I/71XBl40bfiL._AC_UY654_FMwebp_QL65_.jpg']

In [88]:
resolutions = {
    "1x":[],
    "1.5x":[],
    "2x":[],
    "2.5x":[],
    "3x":[],
    "3.5x":[],
    "4x":[],
}

In [102]:
res_regex = re.compile(r'\d\.?\d?x')
res_regex.findall(urls)

['1x', '1.5x', '2x', '2.5x', '3x']

In [39]:
def parse_urls(raw_urls):
    resolutions = {
    "1x":[],
    "1.5x":[],
    "2x":[],
    "2.5x":[],
    "3x":[],
    "3.5x":[],
    "4x":[],
}
    urls = []
    res = []
    web_regex = re.compile(r'https://.*?\.jpg|\.png|\.jpeg|.webp')
    res_regex = re.compile(r'(\d\.?\d?x),')
    
    for u in raw_urls:
        url = web_regex.findall(u)
        resolution = res_regex.findall(u)
        urls.extend(url)
        res.extend(resolution)
    final_urls = [(url, res) for url, res in zip(urls, res)]
    for url, res in final_urls:
        resolutions[res].append(url)
    return resolutions

In [186]:
urls_final = parse_urls(low)
urls_final

{'1x': ['https://m.media-amazon.com/images/I/7117BtmH5BL._AC_UY218_.jpg',
  'https://m.media-amazon.com/images/I/7117BtmH5BL._AC_UY654_FMwebp_QL65_.jpg'],
 '1.5x': ['https://m.media-amazon.com/images/I/7117BtmH5BL._AC_UY327_FMwebp_QL65_.jpg',
  'https://m.media-amazon.com/images/I/91xCCcouWgL._AC_UY218_.jpg'],
 '2x': ['https://m.media-amazon.com/images/I/7117BtmH5BL._AC_UY436_FMwebp_QL65_.jpg',
  'https://m.media-amazon.com/images/I/91xCCcouWgL._AC_UY327_FMwebp_QL65_.jpg'],
 '2.5x': ['https://m.media-amazon.com/images/I/7117BtmH5BL._AC_UY545_FMwebp_QL65_.jpg',
  'https://m.media-amazon.com/images/I/91xCCcouWgL._AC_UY436_FMwebp_QL65_.jpg'],
 '3x': [],
 '3.5x': [],
 '4x': []}

## Getting Urls

In [10]:
def get_images_from_amazon_silent(book, author, num = 3):
    link = "https://www.amazon.in/s?k=" + book+ " " + author + "&i=stripbooks"
    # print("Opening Url")
    driver.get(link)
    # print("Getting Image tags")
    
    i=0
    j=0
    raw_img_srcs = []
    high_image_urls = []
    max_tries = 10
    while True:
        image = driver.find_elements(By.CLASS_NAME, "s-image")[i]
        j+=1
        try:
            if book.lower()[:10] in image.get_attribute("alt").lower():
                # print("Getting image srcs")
                raw_img_srcs.append(image.get_property("srcset"))
                i+=1
                # print("Clicking on image")
                image.click()
                # print("Switching to new window")
                driver.switch_to.window(driver.window_handles[-1])
                # print("Clicking on thumb button")
                try:
                    driver.find_elements(By.CLASS_NAME, "thumb-text")[0].click()
                    # print("Getting High Resolution Image")
                    for el in driver.find_elements(By.CLASS_NAME, "ig-thumb-image"):
                        el.click()
                        high_image_urls.append(driver.find_elements(By.ID, "igImage")[0].get_attribute("src"))
                except:
                    pass
                    # print("No High Resolution Image")
                # print("Closing the Extra Window")
                driver.close()
                # print("Switching to the Main Window")
                driver.switch_to.window(driver.window_handles[0])
        except Exception as e:
            # print(e)
            print("Error in getting image")
            if j>max_tries:
                break
        if i == num:
            break
        if j == max_tries:
            break
    return raw_img_srcs, high_image_urls

In [11]:
high_images = books["High Links"]
low_images = books["Low Links"]

In [46]:
driver.close()

In [47]:
driver = webdriver.Chrome(package)

<ipython-input-47-56b07272da88>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(package)


In [257]:
from life_saving_tools.misc.Notification import Notification
n = Notification()

In [68]:
start = 449
end = 449
j=0
for i in tqdm.tqdm(range(start, end+1), desc="Getting Images"):
    author = authors[i]
    book = book_names[i]
    tick = time.time()
    try:
        low, high = get_images_from_amazon_silent(book, author)
        low_images[i] = parse_urls(low)
        high_images[i] = high
        j=0
    except Exception as e:
        print("Error on", book)
        # raise e
        low = None
        high = None
        low_images[i] = low
        high_images[i] = high
        j+=1
    if j==4:
        tock = time.time()
        j=0
        if tock-tick<5:
            print("Something is Wrong. Restart the driver")
            # n.send_whatsapp_text("Something is Wrong. Restart the driver")
            break
    # if i==end:
    #     n.send_whatsapp_text("Extraction done!")


Getting Images:   0%|          | 0/1 [00:00<?, ?it/s]<ipython-input-68-32b9d3b630f4>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  low_images[i] = parse_urls(low)
<ipython-input-68-32b9d3b630f4>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_images[i] = high
Getting Images: 100%|██████████| 1/1 [00:03<00:00,  3.35s/it]


In [60]:
i

450

In [63]:
book_names[i-1]

'Pride and Prejudice'

In [64]:
high_images[i-1]

[]

In [66]:
low_images[i-2]

{'1x': ['https://m.media-amazon.com/images/I/71STFfTdofL._AC_UY218_.jpg',
  'https://m.media-amazon.com/images/I/71STFfTdofL._AC_UY654_FMwebp_QL65_.jpg',
  'https://m.media-amazon.com/images/I/51BhT7Ey+hL._AC_UY436_FMwebp_QL65_.jpg'],
 '1.5x': ['https://m.media-amazon.com/images/I/71STFfTdofL._AC_UY327_FMwebp_QL65_.jpg',
  'https://m.media-amazon.com/images/I/51BhT7Ey+hL._AC_UY218_.jpg',
  'https://m.media-amazon.com/images/I/51BhT7Ey+hL._AC_UY500_FMwebp_QL65_.jpg'],
 '2x': ['https://m.media-amazon.com/images/I/71STFfTdofL._AC_UY436_FMwebp_QL65_.jpg',
  'https://m.media-amazon.com/images/I/51BhT7Ey+hL._AC_UY327_FMwebp_QL65_.jpg',
  'https://m.media-amazon.com/images/I/81pM4orRvFL._AC_UY218_.jpg'],
 '2.5x': ['https://m.media-amazon.com/images/I/71STFfTdofL._AC_UY545_FMwebp_QL65_.jpg',
  'https://m.media-amazon.com/images/I/81pM4orRvFL._AC_UY327_FMwebp_QL65_.jpg'],
 '3x': [],
 '3.5x': [],
 '4x': []}

In [71]:
l2, h2 = get_images_from_amazon("Pride and Prejudice", "Jane Austen")

Opening Url
Getting Image tags


In [72]:
l = parse_urls(l)
h = h

In [74]:
h

['https://images-na.ssl-images-amazon.com/images/I/819HSIKiWdS.jpg',
 'https://images-na.ssl-images-amazon.com/images/I/81H3klHiJmL.jpg',
 'https://images-na.ssl-images-amazon.com/images/I/81WHo3-GU-L.jpg',
 'https://images-na.ssl-images-amazon.com/images/I/91mNGubXSIL.jpg',
 'https://images-na.ssl-images-amazon.com/images/I/81Fvfz6vWwL.jpg',
 'https://images-na.ssl-images-amazon.com/images/I/81er5HUtctL.jpg',
 'https://images-na.ssl-images-amazon.com/images/I/81ruLJOW-eL.jpg',
 'https://images-na.ssl-images-amazon.com/images/I/61J8NJB9dTL.jpg',
 'https://images-na.ssl-images-amazon.com/images/I/91bEvdgosHL.jpg',
 'https://images-na.ssl-images-amazon.com/images/I/617okC8emiL.jpg',
 'https://images-na.ssl-images-amazon.com/images/I/61JYJHIGqQL.jpg',
 'https://images-na.ssl-images-amazon.com/images/I/61fEeg-78JL.jpg',
 'https://images-na.ssl-images-amazon.com/images/I/614kYodglTL.jpg',
 'https://images-na.ssl-images-amazon.com/images/I/61cUJRiKBBL.jpg',
 'https://images-na.ssl-images-ama

In [76]:
book_names[449]

'Pride and Prejudice'

In [77]:
low_images[449] = l
high_images[449] = h

<ipython-input-77-7a3edddcc4b8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  low_images[449] = l
<ipython-input-77-7a3edddcc4b8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_images[449] = h


In [78]:
low_images[449]

{'1x': ['https://m.media-amazon.com/images/I/819HSIKiWdS._AC_UY218_.jpg',
  'https://m.media-amazon.com/images/I/819HSIKiWdS._AC_UY654_FMwebp_QL65_.jpg'],
 '1.5x': ['https://m.media-amazon.com/images/I/819HSIKiWdS._AC_UY327_FMwebp_QL65_.jpg',
  'https://m.media-amazon.com/images/I/61J8NJB9dTL._AC_UY218_.jpg'],
 '2x': ['https://m.media-amazon.com/images/I/819HSIKiWdS._AC_UY436_FMwebp_QL65_.jpg',
  'https://m.media-amazon.com/images/I/61J8NJB9dTL._AC_UY327_FMwebp_QL65_.jpg'],
 '2.5x': ['https://m.media-amazon.com/images/I/819HSIKiWdS._AC_UY545_FMwebp_QL65_.jpg',
  'https://m.media-amazon.com/images/I/61J8NJB9dTL._AC_UY436_FMwebp_QL65_.jpg'],
 '3x': [],
 '3.5x': [],
 '4x': []}

In [79]:
books["Low Links"] = low_images
books["High Links"] = high_images

In [80]:
books.head()

,Book Name,Author,ISBN 13,ISBN,Number of Pages,Average Rating,Total Number of Ratings,Publication Date,Date Started,Date Completed,Name,Series,Volume,Goodreads Url,Description,Genre,Image Url,Low Links,High Links
0,The Lightning Tree,Patrick Rothfuss,NaN,NaN,93,4.00,2156,2014-01-01,2022-04-16,2022-04-16,The Lightning Tree,Standalone,1,https://www.goodreads.com/book/show/55262194,The Lightning Tree is a companion short story ...,"['Fantasy', 'Fiction', 'Short Stories', 'Fanta...",https://i.gr-assets.com/images/S/compressed.ph...,"{'1x': [], '1.5x': [], '2x': [], '2.5x': [], '...",[]
1,What If? Serious Scientific Answers to Absurd ...,Randall Munroe,9.780544e+12,0544272994,303,4.14,151303,2014-09-02,NaN,NaN,What If? Serious Scientific Answers to Absurd ...,What If?,1,https://www.goodreads.com/book/show/21413662,Randall Munroe left NASA in 2005 to start up h...,"['Nonfiction', 'Science', 'Humor', 'Audiobook'...",https://i.gr-assets.com/images/S/compressed.ph...,NaN,NaN
2,"Leviathan Falls (The Expanse, #9)",James S.A. Corey,9.780357e+12,0356510395,528,4.54,31066,2021-11-30,NaN,NaN,Leviathan Falls,The Expanse,9,https://www.goodreads.com/book/show/58783427,"The Laconian Empire has fallen, setting the th...","['Science Fiction', 'Fiction', 'Space', 'Space...",NaN,{'1x': ['https://m.media-amazon.com/images/I/8...,[]
3,"Tiamat's Wrath (The Expanse, #8)",James S.A. Corey,9.780316e+12,0316332879,534,4.57,47842,2019-03-26,NaN,NaN,Tiamat's Wrath,The Expanse,8,https://www.goodreads.com/book/show/28335698,Thirteen hundred gates have opened to solar sy...,"['Science Fiction', 'Fiction', 'Space', 'Space...",https://i.gr-assets.com/images/S/compressed.ph...,{'1x': ['https://m.media-amazon.com/images/I/8...,['https://images-na.ssl-images-amazon.com/imag...
4,"Persepolis Rising (The Expanse, #7)",James S.A. Corey,9.780316e+12,0316332828,608,4.37,53386,2017-12-05,2022-06-02,NaN,Persepolis Rising,The Expanse,7,https://www.goodreads.com/book/show/28335696,In the thousand-sun network of humanity's expa...,"['Science Fiction', 'Fiction', 'Space', 'Space...",https://i.gr-assets.com/images/S/compressed.ph...,{'1x': ['https://m.media-amazon.com/images/I/9...,['https://images-na.ssl-images-amazon.com/imag...


In [82]:
books.to_csv("Books_Final.csv", index=False)

## Downloading Images from URL

In [87]:
images = high_images[449]

In [89]:
image_url =images[0]
response = requests.get(image_url)
with open(".temp.jpg", "wb") as f:
    f.write(response.content)

img = Image.open(".temp.jpg")
# img.show()
# img = Image.open(BytesIO(response.content))
w, h = img.size
print(f"Image shape is {w}x{h}")

Image shape is 2367x2560


In [133]:
from PIL import features
print(features.check_module('webp'))

False
